In [2]:
!pip install "pylibCZIrw~=4.0" "cztile>=0.0,<1.0" matplotlib tqdm scikit-image pooch
!pip install pillow

In [3]:
from pylibCZIrw import czi as pyczi
import numpy as np
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it

C:\Users\schatzm\Anaconda3\envs\libczi_cutter\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
czifile = "F:\\PROJECTS\\Axioscan_Analysis\\Vojtech_Abraham\\230518_ML_pyl_OB\\2023_05_18_Pyl-OB_001.czi"

In [26]:
with pyczi.open_czi(czifile) as czidoc:
    # get the image dimensions as an dictionary, where the key identifies the dimension
    total_bounding_box = czidoc.total_bounding_box

print(total_bounding_box)

{'T': (0, 1), 'Z': (0, 49), 'C': (0, 1), 'X': (-161280, -56000), 'Y': (8640, 35760)}


In [19]:
total_bounding_box['Z'][1]

49

In [20]:
with pyczi.open_czi(czifile) as czidoc:
    # get the total bounding box for all scenes
    total_bounding_rectangle = czidoc.total_bounding_rectangle

print(total_bounding_rectangle)

Rectangle(x=-161280, y=8640, w=105280, h=27120)


In [21]:
## helpers
from PIL import Image
def save_roi_as_image(roi, output_path):
    # Convert numpy array to PIL Image
    roi_image = Image.fromarray(roi, 'RGB')
    
    # Save the ROI image
    roi_image.save(output_path)


In [22]:
#!pip install tifffile

In [48]:
import tifffile
import os

def libczi_cutter(czifile,output_folder,name,roi_width,roi_height):

    # print("Output:")
    # print(output_folder)
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # import the "tiling strategy" from the cztile package
    from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D
    
    # create a "tile" by specifying the desired tile dimension and the
    # minimum required overlap between tiles (depends on the processing)
    tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(
        total_tile_width=roi_width,
        total_tile_height=roi_height,
        min_border_width=0
    )
    
    
    # open a CZI instance to read and in parallel one to write
    with pyczi.open_czi(czifile) as czidoc_r:
        # get bounding box
        total_bounding_box = czidoc_r.total_bounding_box

        # print(total_bounding_box)

        # read metadata
        md_xml = czidoc_r.metadata #raw_metadata
        
        # get the size of the bounding rectange for the scence
        tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[0])
        tile_counter=0
        for tile in tqdm(tiles):
            # create empty RGB z stack
            BGR_stack=np.zeros([total_bounding_box['Z'][1], 3, roi_height, roi_width])
            # go over Z from 0 to  total_bounding_box['Z'][1]
            for z in range(0,total_bounding_box['Z'][1]):
            # read a specific tile from the CZI using the roi parameter
                tile2d = czidoc_r.read(
                    plane={"C": 0, "Z":z, "T":0},
                    roi=(
                        tile.roi.x,
                        tile.roi.y,
                        tile.roi.w,
                        tile.roi.h
                    )
                )
                # print(tile2d.shape)
                # print([tile.roi.x,
                #         tile.roi.y,
                #         tile.roi.w,
                #         tile.roi.h])
    
                # has to be Z C Y X
                BGR_stack[z,:,:,:]= np.transpose(tile2d, (2, 0, 1))
                
            # if tile_counter==3:
            #     break
            
            # convert BGR, save RGB_stack
            RGB_stack=np.zeros([total_bounding_box['Z'][1], 3, roi_height, roi_width])
            RGB_stack[:,0,:,:]=BGR_stack[:,2,:,:]
            RGB_stack[:,1,:,:]=BGR_stack[:,1,:,:]
            RGB_stack[:,2,:,:]=BGR_stack[:,0,:,:]

            img_name = name + '_X-{}_Y-{}_stack-{}.tif'.format(tile.roi.x,tile.roi.y,tile_counter)

            output_file = output_folder + "\\" + img_name
            tifffile.imwrite(output_file, RGB_stack.astype('uint8'), imagej=False , photometric='rgb', metadata=md_xml)
            tile_counter = tile_counter +1

In [ ]:
path = "F:\\PROJECTS\\Axioscan_Analysis\\Vojtech_Abraham\\230518_ML_pyl_OB\\"

roi_width = 1920
roi_height = 1600

skip = []

# files = os.listdir("../../230518_ML_pyl_OB/")
files = os.listdir(path)
# files = os.listdir("../../test/")

print(len(files))

for file in files:
    if file not in skip:
        # Get the file extension
        file_extension = os.path.splitext(file)[1]
        file_w = os.path.splitext(file)[0]

        # Check if the file extension is ".czi"
        if file_extension.lower() == ".czi":
            # print(file_w)
            fpath = f'F:\\PROJECTS\\Axioscan_Analysis\\Vojtech_Abraham\\libczi_output\\{file_w}'
            #if os.path.isfile(fpath)
            print(fpath)

            libczi_cutter(os.path.join(path,file),fpath,file,roi_width,roi_height)
            print(file)
            print("Saved")
            print("_______________________")


53
F:\PROJECTS\Axioscan_Analysis\Vojtech_Abraham\libczi_output\2023_05_18_Pyl-OB_001


  1%|█▏                                                                             | 14/935 [02:21<3:56:55, 15.44s/it]